# AED exploration and opening hours

In [3]:
import openrouteservice
from openrouteservice import client
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import time
import geopy.distance

import pandas as pd
import numpy as np
import openrouteservice
from openrouteservice import client
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import time
import geopy.distance


import folium
import geopandas as gpd
from shapely.geometry import Point
import random

In [315]:
AEDs = gpd.read_file('C:/Users/leonw/OneDrive - KU Leuven/Documents/GitHub/AED-Route-Optimization-MDA-Project/Data/temp.gpkg', layer='AED_data')

In [316]:
AEDs

,lon,lat,geometry
0,4.737563,50.847642,"POLYGON ((4.72665 50.84916, 4.72657 50.84817, ..."
1,4.696771,50.882662,"POLYGON ((4.68680 50.88487, 4.68743 50.88203, ..."
2,4.705893,50.891915,"POLYGON ((4.69766 50.88972, 4.69971 50.88766, ..."
3,4.718684,50.893646,"POLYGON ((4.71665 50.89818, 4.71672 50.89643, ..."
4,4.693191,50.869499,"POLYGON ((4.68327 50.86743, 4.68455 50.86551, ..."
...,...,...,...
134,4.676583,50.865563,"POLYGON ((4.67218 50.86315, 4.67219 50.86305, ..."
135,4.676583,50.865563,"POLYGON ((4.67218 50.86315, 4.67219 50.86305, ..."
136,4.711970,50.868075,"POLYGON ((4.70027 50.86808, 4.70228 50.86644, ..."
137,4.748855,50.870141,"POLYGON ((4.73898 50.87222, 4.74184 50.86737, ..."


In [301]:
AEDs.head()

,lon,lat,geometry
0,4.737563,50.847642,"POLYGON ((4.72665 50.84916, 4.72657 50.84817, ..."
1,4.696771,50.882662,"POLYGON ((4.68680 50.88487, 4.68743 50.88203, ..."
2,4.705893,50.891915,"POLYGON ((4.69766 50.88972, 4.69971 50.88766, ..."
3,4.718684,50.893646,"POLYGON ((4.71665 50.89818, 4.71672 50.89643, ..."
4,4.693191,50.869499,"POLYGON ((4.68327 50.86743, 4.68455 50.86551, ..."


In [28]:
# Target coordinates
target_lon, target_lat = 4.69932, 50.88762
x = AEDs.copy()
# Compute Euclidean distance (in degrees)
x['distance'] = np.sqrt((AEDs['lon'] - target_lon)**2 + (AEDs['lat'] - target_lat)**2)

# Get the indices of the 5 closest locations
closest_indices = x.nsmallest(5, 'distance').index.tolist()

print(closest_indices)

[118, 87, 1, 103, 11]


In [11]:
AEDs.iloc[nearest_index]

lon                                                  4.729686
lat                                                 50.886956
geometry    POLYGON ((4.72105 50.890037, 4.719638 50.88527...
Name: 100, dtype: object

In [9]:
nearest_index

88

In [ ]:
import folium
import geopandas as gpd
from shapely.geometry import Point
import openrouteservice

def find_isochrone(coordinate, profile="foot-walking", threshold=600):
    isochrones_parameters = {
        'locations': coordinate,
        'profile': profile,
        'range_type': 'time',
        'range': [threshold]  # 10 minutes (600 seconds)
    }

    ors_client_custom = openrouteservice.Client(key='5b3ce3597851110001cf624802e069d6633748a5ae4e9842334f1dc2')
    isochrone = ors_client_custom.isochrones(**isochrones_parameters)
    
    return isochrone

# Define the patient's location
coordinate = [[4.69932, 50.88762]]
patients = (4.69932, 50.88762)
# Fetch the isochrone for the patient
iso_patient = find_isochrone(coordinate)




In [212]:
iso_patient_vector = find_isochrone(coordinate, profile="driving-car")


In [ ]:
iso_patient_3 = find_isochrone(coordinate, threshold = 840)

In [121]:
gdf = AEDs.iloc[[1,2,87,11]].copy()
gdf.loc[:,'AED'] = list(zip(gdf['lon'], gdf['lat']))
gdf.head()


,lon,lat,geometry,AED
1,4.696771,50.882662,"POLYGON ((4.68680 50.88487, 4.68743 50.88203, ...","(4.6967705, 50.8826624)"
2,4.705893,50.891915,"POLYGON ((4.69766 50.88972, 4.69971 50.88766, ...","(4.7058934, 50.89191539999999)"
87,4.704496,50.885961,"POLYGON ((4.69471 50.88690, 4.69441 50.88522, ...","(4.704496199999999, 50.8859605)"
11,4.701441,50.882289,"POLYGON ((4.69201 50.88066, 4.69260 50.87982, ...","(4.701441099999999, 50.88228909999999)"


In [303]:
gdf_AED = AEDs.copy()
gdf_AED.loc[:,'AED'] = list(zip(gdf_AED['lon'], gdf_AED['lat']))
gdf_AED.head()
# Convert AED column to Points if not already in geometry format
if not isinstance(gdf_AED['AED'].iloc[0], Point):
    gdf_AED['AED'] = gdf_AED['AED'].apply(lambda coords: Point(coords) if isinstance(coords, (list, tuple)) else coords)

# Set AED as the active geometry column in the GeoDataFrame
gdf_AED = gdf_AED.set_geometry('AED')

In [305]:
gdf_AED

,lon,lat,geometry,AED
0,4.737563,50.847642,"POLYGON ((4.72665 50.84916, 4.72657 50.84817, ...",POINT (4.73756 50.84764)
1,4.696771,50.882662,"POLYGON ((4.68680 50.88487, 4.68743 50.88203, ...",POINT (4.69677 50.88266)
2,4.705893,50.891915,"POLYGON ((4.69766 50.88972, 4.69971 50.88766, ...",POINT (4.70589 50.89192)
3,4.718684,50.893646,"POLYGON ((4.71665 50.89818, 4.71672 50.89643, ...",POINT (4.71868 50.89365)
4,4.693191,50.869499,"POLYGON ((4.68327 50.86743, 4.68455 50.86551, ...",POINT (4.69319 50.86950)
...,...,...,...,...
134,4.676583,50.865563,"POLYGON ((4.67218 50.86315, 4.67219 50.86305, ...",POINT (4.67658 50.86556)
135,4.676583,50.865563,"POLYGON ((4.67218 50.86315, 4.67219 50.86305, ...",POINT (4.67658 50.86556)
136,4.711970,50.868075,"POLYGON ((4.70027 50.86808, 4.70228 50.86644, ...",POINT (4.71197 50.86807)
137,4.748855,50.870141,"POLYGON ((4.73898 50.87222, 4.74184 50.86737, ...",POINT (4.74886 50.87014)


In [306]:

import folium
import geopandas as gpd
from shapely.geometry import Point

# Assuming 'gdf' is your GeoDataFrame with 'AED' as points and 'Iso' as isochrones (polygons)

#


# Convert AED column to Points if not already in geometry format
if not isinstance(gdf['AED'].iloc[0], Point):
    gdf['AED'] = gdf['AED'].apply(lambda coords: Point(coords) if isinstance(coords, (list, tuple)) else coords)

# Set AED as the active geometry column in the GeoDataFrame
gdf = gdf.set_geometry('AED')

# Initialize a folium map centered around the average AED location
m = folium.Map(location=[gdf['AED'].y.mean(), gdf['AED'].x.mean()], zoom_start=13)

# Add isochrones (polygons) to the map
# for _, row in gdf.iterrows():
#     folium.GeoJson(
#         row['Iso'],
#         style_function=lambda x: {
#             'fillColor': 'blue',
#             'color': 'blue',
#             'weight': 2,
#             'fillOpacity': 0.3
#         }
#     ).add_to(m)

# Add AED points to the map
for _, row in gdf_AED.iterrows():
    folium.Marker(
        location=[row['AED'].y, row['AED'].x],
        popup="AED",
        icon=folium.Icon(color='red', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Save and display the map
m.save("aed_isochrones_map.html")

In [307]:
m

In [17]:
import folium
import geopandas as gpd
import random
from shapely.geometry import Point
import openrouteservice
import numpy as np
def generate_random_points(center, num_points=10, radius=500):
    """Generate random points around a given center within a radius in meters."""
    lat, lon = center
    earth_radius = 6371000  # meters

    points = []
    for _ in range(num_points):
        # Generate random distances within the radius
        rand_dist = random.uniform(0, radius)
        rand_angle = random.uniform(0, 2 * np.pi)

        # Convert meters to degrees (approximation)
        delta_lat = (rand_dist / earth_radius) * (180 / np.pi)
        delta_lon = (rand_dist / (earth_radius * np.cos(np.radians(lat)))) * (180 / np.pi)

        new_lat = lat + delta_lat * np.cos(rand_angle)
        new_lon = lon + delta_lon * np.sin(rand_angle)

        points.append((new_lat, new_lon))

    return points

In [86]:

def generate_random_points_outside(center, num_points=10, min_radius=1100, max_radius=3000):
    """Generate random points around a given center with distances between min_radius and max_radius (in meters)."""
    lat, lon = center
    earth_radius = 6371000  # meters

    points = []
    for _ in range(num_points):
        # Generate random distances between min_radius and max_radius
        rand_dist = random.uniform(min_radius, max_radius)
        rand_angle = random.uniform(0, 2 * np.pi)

        # Convert meters to degrees (approximation)
        delta_lat = (rand_dist / earth_radius) * (180 / np.pi)
        delta_lon = (rand_dist / (earth_radius * np.cos(np.radians(lat)))) * (180 / np.pi)

        new_lat = lat + delta_lat * np.cos(rand_angle)
        new_lon = lon + delta_lon * np.sin(rand_angle)

        points.append((new_lat, new_lon))

    return points

In [87]:
# Generate points between 500m and 2000m away
points = generate_random_points_outside(center=(4.69932, 50.88762), num_points=30)

In [88]:
random_gdf_outside = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in points])

In [ ]:
# Generate random points around the patient
# random_points = generate_random_points(patients, num_points=10, radius=1000)
# # Convert random points to GeoDataFrame
# random_gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in random_points])

In [20]:
random_points = [(4.699494020757639, 50.89538254337843),
 (4.696070395010916, 50.895973989960524),
 (4.6988937307183765, 50.88928789041148),
 (4.694073171314823, 50.88456620138093),
 (4.69423913260052, 50.89444381614162),
 (4.697383285811491, 50.88792467151592),
 (4.6983404708145855, 50.891136735680725),
 (4.699814863109981, 50.88963071924299),
 (4.699446605146951, 50.8861053526476586),
 (4.706083456934369, 50.88740452107823),
 (4.70814863109981, 50.89963071924299),
 (4.699846605146951, 50.8851053526476586),
 (4.709083456934369, 50.88749452107823)]

In [21]:
random_gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in random_points])

In [186]:
amb_loc = pd.read_parquet("Data/ambulance_locations.parquet.gzip")
Ambulances = amb_loc[amb_loc['province']=='Vlaams-Brabant']
# Only keep rows that contain 30 as all postla code of Leuven start with 30
Ambulances = Ambulances[Ambulances['departure_location'].str.contains('30')]
Ambulances = Ambulances.reset_index(drop = True)
df = pd.DataFrame()
df['Name'] = Ambulances['departure_location_number']
df['longitude'] = Ambulances['longitude']
df['latitude'] = Ambulances['latitude']

In [60]:
df

,Name,longitude,latitude
0,25403,4.724344,50.869655
1,3521,4.695398,50.878535
2,3820,4.098320,50.868004
3,26401,4.472760,50.879821
4,23001,4.665862,50.877460
5,23005,4.527038,50.777013
6,23003,4.937178,50.823648
7,24006,4.500188,50.875499
8,26301,4.699833,50.872354
9,28801,4.673199,50.878079


In [65]:
df_ambulance = df.iloc[1].copy()

In [ ]:
gdf['AED'] = gdf['AED'].apply(lambda coords: Point(coords) if isinstance(coords, (list, tuple)) else coords)

# Set AED as the active geometry column in the GeoDataFrame
gdf = gdf.set_geometry('AED')

In [ ]:
# Randomly generate numbers with a decline_rate. 
# Data frame with all the possible responders and their probability to accept the call. 1 for acceptance
acceptance = list(random.choice([0,1], p=[decline_rate, (1-decline_rate)], size = len(possible_responder)))
df_acceptance = pd.DataFrame({'Responder_loc' : possible_responder,
                                'Probability' : acceptance})
        

In [310]:
icon_path = "C:/Users/leonw/Downloads/Automated_External_Defibrillator_(symbol).svg.png"
# Create the custom icon
custom_icon = folium.CustomIcon(
    icon_image=icon_path,
    icon_size=(30, 30),  # Adjust size as needed
    icon_anchor=(20, 20)  # Center the icon
)

In [22]:
from selenium import webdriver
from PIL import Image
import time


In [213]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15.4, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient_vector,
    style_function=lambda x: {
        'fillColor': 'orange',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)

# # Add the patient marker (in red)
folium.CircleMarker(
    location=[patients[1], patients[0]],
    radius=7,  # Bigger circle
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(m)


folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa', icon_color='black')  # Red icon on white marker
).add_to(m)

# Add AEDs markers (red)
folium.Marker(
    location=[df_ambulance['latitude'], df_ambulance['longitude']],
    popup="Vector",
    icon=folium.Icon(color='orange', icon='truck-medical', prefix='fa')
).add_to(m)
m

In [90]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15.4, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)

# # Add the patient marker (in red)
folium.CircleMarker(
    location=[patients[1], patients[0]],
    radius=7,  # Bigger circle
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(m)


folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa', icon_color='black')  # Red icon on white marker
).add_to(m)


# Add random points (blue)
for lat, lon in random_points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

# Add random points (blue)
for lat, lon in points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

m

In [93]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=14, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
# folium.GeoJson(
#     iso_patient,
#     style_function=lambda x: {
#         'fillColor': 'red',
#         'color': 'red',
#         'weight': 2,
#         'fillOpacity': 0.3
#     }
# ).add_to(m)

# Add patient marker (red)
folium.CircleMarker(
    location=[patients[1], patients[0]],
    radius=7,  # Bigger circle
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(m)

# # Add the patient marker (in green)
# folium.Marker(
#     location=[patients[1], patients[0]],
#     popup="Patient",
#     icon=folium.Icon(color='green', icon='hospital-user', prefix='fa')
# ).add_to(m)


# Add the patient's isochrone (in red)
# folium.GeoJson(
#     iso_patient_vector,
#     style_function=lambda x: {
#         'fillColor': 'orange',
#         'color': 'red',
#         'weight': 2,
#         'fillOpacity': 0.3
#     }
# ).add_to(m)



folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa', icon_color='black')  # Red icon on white marker
).add_to(m)

# Add AEDs markers (red)
folium.Marker(
    location=[df_ambulance['latitude'], df_ambulance['longitude']],
    popup="Vector",
    icon=folium.Icon(color='orange', icon='truck-medical', prefix='fa')
).add_to(m)

# Add Vector markers (red)
for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup="Vector",
        icon=folium.Icon(color='orange', icon='truck-medical', prefix='fa')
    ).add_to(m)

# Add AEDs markers (red)
for _, row in AEDs.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points outside (blue)
for lat, lon in points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

# Add random points (blue)
for lat, lon in random_points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

# Show the map
m

In [109]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=14, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)

# Add patient marker (red)
folium.CircleMarker(
    location=[patients[1], patients[0]],
    radius=7,  # Bigger circle
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(m)

# # Add the patient marker (in green)
# folium.Marker(
#     location=[patients[1], patients[0]],
#     popup="Patient",
#     icon=folium.Icon(color='green', icon='hospital-user', prefix='fa')
# ).add_to(m)


# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient_vector,
    style_function=lambda x: {
        'fillColor': 'orange',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)



folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa', icon_color='black')  # Red icon on white marker
).add_to(m)

# Add AEDs markers (red)
folium.Marker(
    location=[df_ambulance['latitude'], df_ambulance['longitude']],
    popup="Vector",
    icon=folium.Icon(color='orange', icon='truck-medical', prefix='fa')
).add_to(m)

# Add Vector markers (red)
for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup="Vector",
        icon=folium.Icon(color='orange', icon='truck-medical', prefix='fa')
    ).add_to(m)

# Add AEDs markers (red)
for _, row in AEDs.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points outside (blue)
for lat, lon in points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

# Add random points (blue)
for lat, lon in random_points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

# Show the map
m

In [322]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15.4, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'darkred',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)



# Add isochrones for AEDs (in blue)
for _, row in gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'green',
            'weight': 2,
            'fillOpacity': 0.3
        }
    ).add_to(m)

# Add AED markers (in red)
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row['AED'].y, row['AED'].x],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points (blue)
for lat, lon in random_points:
    folium.Marker(
        location=[lon, lat],
        popup="Random Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

folium.CircleMarker(
    location=[patients[1], patients[0]],
    radius=7,  # Bigger circle
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(m)

# # Add the patient marker (in green)
# folium.Marker(
#     location=[patients[1], patients[0]],
#     popup="Patient",
#     icon=folium.Icon(color='green', icon='hospital-user', prefix='fa')
# ).add_to(m)

folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa', icon_color='black')  # Red icon on white marker
).add_to(m)
# Show the map

m

In [38]:
# Filter points inside the isochrone polygon
# Convert Isochrone to Polygon
from shapely.geometry import Point, Polygon, shape
iso_polygon = shape(iso_patient['features'][0]['geometry'])



In [39]:
gdf_iso_polygon = gpd.GeoDataFrame({'geometry': [geometry.Polygon(iso_patient['features'][0]['geometry']['coordinates'][0])]})
gdf_iso_polygon

,geometry
0,"POLYGON ((4.68929 50.88522, 4.69125 50.88321, ..."


In [40]:
random_points

[(4.699494020757639, 50.89538254337843),
 (4.696070395010916, 50.895973989960524),
 (4.6988937307183765, 50.88928789041148),
 (4.694073171314823, 50.88456620138093),
 (4.69423913260052, 50.89444381614162),
 (4.697383285811491, 50.88792467151592),
 (4.6983404708145855, 50.891136735680725),
 (4.699814863109981, 50.88963071924299),
 (4.699446605146951, 50.88610535264766),
 (4.706083456934369, 50.88740452107823),
 (4.70814863109981, 50.89963071924299),
 (4.699846605146951, 50.885105352647656),
 (4.709083456934369, 50.88749452107823)]

In [ ]:

filter_points = list([geometry.Point(random_points[i][0],random_points[i][1]) for i in range(len(random_points))])
filter_points


[<POINT (4.699 50.895)>,
 <POINT (4.696 50.896)>,
 <POINT (4.699 50.889)>,
 <POINT (4.694 50.885)>,
 <POINT (4.694 50.894)>,
 <POINT (4.697 50.888)>,
 <POINT (4.698 50.891)>,
 <POINT (4.7 50.89)>,
 <POINT (4.699 50.886)>,
 <POINT (4.706 50.887)>,
 <POINT (4.708 50.9)>,
 <POINT (4.7 50.885)>,
 <POINT (4.709 50.887)>]

In [ ]:
df

,Name,longitude,latitude
0,25403,4.724344,50.869655
1,3521,4.695398,50.878535
2,3820,4.098320,50.868004
3,26401,4.472760,50.879821
4,23001,4.665862,50.877460
5,23005,4.527038,50.777013
6,23003,4.937178,50.823648
7,24006,4.500188,50.875499
8,26301,4.699833,50.872354
9,28801,4.673199,50.878079


In [188]:
coordinate_tuples = [[(point)] for point in filter_points if point.within(gdf_iso_polygon_Vec.loc[0,'geometry'])]
df_filter_points = pd.DataFrame(coordinate_tuples, columns = ['coordinates'])

In [101]:
df_filter_points.loc[1]['coordinates'].y

50.88456620138093

In [189]:
df_filter_points

,coordinates
0,POINT (4.699494020757639 50.89538254337843)
1,POINT (4.696070395010916 50.895973989960524)
2,POINT (4.6988937307183765 50.88928789041148)
3,POINT (4.694073171314823 50.88456620138093)
4,POINT (4.69423913260052 50.89444381614162)
5,POINT (4.697383285811491 50.88792467151592)
6,POINT (4.6983404708145855 50.891136735680725)
7,POINT (4.699814863109981 50.88963071924299)
8,POINT (4.699446605146951 50.88610535264766)
9,POINT (4.706083456934369 50.88740452107823)


In [54]:
df_filter_accepted = df_filter_points.iloc[0:6].copy()

In [55]:
df_filter_accepted

,coordinates
0,POINT (4.6988937307183765 50.88928789041148)
1,POINT (4.694073171314823 50.88456620138093)
2,POINT (4.697383285811491 50.88792467151592)
3,POINT (4.6983404708145855 50.891136735680725)
4,POINT (4.699814863109981 50.88963071924299)
5,POINT (4.699446605146951 50.88610535264766)


In [ ]:
AEDs_filtered_points = AEDs.copy()
AEDs_filtered_points['coordinates'] = AEDs_filtered_points.apply(lambda row: geometry.Point(row['lon'], row['lat']), axis=1)


In [103]:
coordinate_tuples_AED = [[(point)] for point in AEDs_filtered_points['coordinates'] if point.within(gdf_iso_polygon.loc[0,'geometry'])]
df_filter_AED = pd.DataFrame(coordinate_tuples_AED, columns = ['coordinates'])

In [104]:
df_filter_AED

,coordinates
0,POINT (4.6967705 50.8826624)
1,POINT (4.7058934 50.89191539999999)
2,POINT (4.701441099999999 50.88228909999999)
3,POINT (4.7031832 50.8816687)
4,POINT (4.7054199 50.8830582)
5,POINT (4.7058934 50.89191539999999)
6,POINT (4.704496199999999 50.8859605)
7,POINT (4.6967705 50.8826624)
8,POINT (4.7007067 50.8851678)


In [193]:
gdf_iso_polygon_Vec = gpd.GeoDataFrame({'geometry': [geometry.Polygon(iso_patient_vector['features'][0]['geometry']['coordinates'][0])]})
gdf_iso_polygon_Vec

,geometry
0,"POLYGON ((4.55493 50.86731, 4.56009 50.86697, ..."


In [195]:
df

,Name,longitude,latitude
0,25403,4.724344,50.869655
1,3521,4.695398,50.878535
2,3820,4.098320,50.868004
3,26401,4.472760,50.879821
4,23001,4.665862,50.877460
5,23005,4.527038,50.777013
6,23003,4.937178,50.823648
7,24006,4.500188,50.875499
8,26301,4.699833,50.872354
9,28801,4.673199,50.878079


In [201]:
Vec_filtered_points = df.copy()
Vec_filtered_points['coordinates'] = Vec_filtered_points.apply(lambda row: geometry.Point(row['longitude'], row['latitude']), axis=1)
Vec_filtered_points

,Name,longitude,latitude,coordinates
0,25403,4.724344,50.869655,POINT (4.7243436 50.8696549)
1,3521,4.695398,50.878535,POINT (4.6953981 50.8785355)
2,3820,4.098320,50.868004,POINT (4.0983199 50.8680038)
3,26401,4.472760,50.879821,POINT (4.4727595 50.8798215)
4,23001,4.665862,50.877460,POINT (4.6658621 50.8774603)
5,23005,4.527038,50.777013,POINT (4.5270384 50.7770126)
6,23003,4.937178,50.823648,POINT (4.9371776 50.8236483)
7,24006,4.500188,50.875499,POINT (4.500188 50.8754995)
8,26301,4.699833,50.872354,POINT (4.6998331 50.8723536)
9,28801,4.673199,50.878079,POINT (4.6731993 50.8780791)


In [202]:
coordinate_tuples_Vec = [[(point)] for point in Vec_filtered_points['coordinates'] if point.within(gdf_iso_polygon_Vec.loc[0,'geometry'])]
df_filter_Vec = pd.DataFrame(coordinate_tuples_Vec, columns = ['coordinates'])
df_filter_Vec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   coordinates  5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes


In [203]:
df_filter_Vec

,coordinates
0,POINT (4.7243436 50.8696549)
1,POINT (4.6953981 50.8785355)
2,POINT (4.6658621 50.8774603)
3,POINT (4.6998331 50.8723536)
4,POINT (4.6731993 50.8780791)


In [110]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=14, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)

# Add patient marker (red)
folium.CircleMarker(
    location=[patients[1], patients[0]],
    radius=7,  # Bigger circle
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(m)

# # Add the patient marker (in green)
# folium.Marker(
#     location=[patients[1], patients[0]],
#     popup="Patient",
#     icon=folium.Icon(color='green', icon='hospital-user', prefix='fa')
# ).add_to(m)


# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient_vector,
    style_function=lambda x: {
        'fillColor': 'orange',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.3
    }
).add_to(m)



folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa', icon_color='black')  # Red icon on white marker
).add_to(m)

# Add AEDs markers (red)
folium.Marker(
    location=[df_ambulance['latitude'], df_ambulance['longitude']],
    popup="Vector",
    icon=folium.Icon(color='orange', icon='truck-medical', prefix='fa')
).add_to(m)

# Add Vector markers (red)
for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup="Vector",
        icon=folium.Icon(color='orange', icon='truck-medical', prefix='fa')
    ).add_to(m)

# Add AED markers (in red)
for _, row in df_filter_AED.iterrows():
    folium.Marker(
        location=[row['coordinates'].y, row['coordinates'].x],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
for _,row in df_filter_points.iterrows():
    folium.Marker(
        location=[row['coordinates'].y, row['coordinates'].x],
        popup="Valid Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)
# Show the map
m

In [130]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15.4, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.15
    }
).add_to(m)

# # Add the patient marker (in red)
folium.CircleMarker(
    location=[patients[1], patients[0]],
    radius=7,  # Bigger circle
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(m)


folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa', icon_color='black')  # Red icon on white marker
).add_to(m)

# Add isochrones for AEDs (in blue)
for _, row in gdf_open.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'green',
            'weight': 2,
            'fillOpacity': 0.15
        }
    ).add_to(m)

# Add AED markers (in red)
# for _, row in df_filter_AED.iterrows():
#     folium.Marker(
#         location=[row['coordinates'].y, row['coordinates'].x],
#         popup="AED",
#         icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
#     ).add_to(m)

# Add AED markers (in red)
for _, row in gdf_open.iterrows():
    folium.Marker(
        location=[row['AED'].y, row['AED'].x],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
# for _,row in df_filter_points.iterrows():
#     folium.Marker(
#         location=[row['coordinates'].y, row['coordinates'].x],
#         popup="Valid Point",
#         icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
#     ).add_to(m)
# Show the map

for _,row in df_filter_accepted.iterrows():
    folium.Marker(
        location=[row['coordinates'].y, row['coordinates'].x],
        popup="Valid Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)


m

# Now route

In [314]:
AEDs

,lon,lat,geometry
0,4.737563,50.847642,"POLYGON ((4.72665 50.84916, 4.72657 50.84817, ..."
1,4.696771,50.882662,"POLYGON ((4.68680 50.88487, 4.68743 50.88203, ..."
2,4.705893,50.891915,"POLYGON ((4.69766 50.88972, 4.69971 50.88766, ..."
3,4.718684,50.893646,"POLYGON ((4.71665 50.89818, 4.71672 50.89643, ..."
4,4.693191,50.869499,"POLYGON ((4.68327 50.86743, 4.68455 50.86551, ..."
...,...,...,...
134,4.676583,50.865563,"POLYGON ((4.67218 50.86315, 4.67219 50.86305, ..."
135,4.676583,50.865563,"POLYGON ((4.67218 50.86315, 4.67219 50.86305, ..."
136,4.711970,50.868075,"POLYGON ((4.70027 50.86808, 4.70228 50.86644, ..."
137,4.748855,50.870141,"POLYGON ((4.73898 50.87222, 4.74184 50.86737, ..."


In [112]:

# Initialize ORS Client
ORS_API_KEY = '5b3ce3597851110001cf624802e069d6633748a5ae4e9842334f1dc2'
ors_client = openrouteservice.Client(key=ORS_API_KEY)

In [4]:
def directions(coordinates, profile = 'foot-walking', sleep = 0):
        
    client = openrouteservice.Client(base_url='http://ec2-3-76-12-187.eu-central-1.compute.amazonaws.com:8080/ors')
    time.sleep(sleep)
    route = client.directions(coordinates=coordinates,
                                   profile=profile,
                                   format='geojson',
                                   validate=False)
    # empty dictionary to be filled later
    route_dict = {}
    # check if all coordinates are the same
    # If so duration would be empty thus it is set to 0
    if coordinates.count(coordinates[0]) == len(coordinates):
        route_dict['duration'] = 0
        route_dict['route'] = route
        route_dict['coordinates'] = route.get('features')[0]['geometry']['coordinates']
    else:
        route_dict['duration'] = route.get('features')[0]['properties']['summary']['duration']
        route_dict['route'] = route
        route_dict['coordinates'] = route.get('features')[0]['geometry']['coordinates']
    return route_dict

In [5]:
route = directions([[8.680678,49.407841], [8.693779,49.407558]])

ApiError: 500 ({'error': {'code': 2099, 'message': 'Unable to get an appropriate route profile for RoutePreference = foot-walking'}, 'info': {'engine': {'build_date': '2024-03-21T13:55:54Z', 'version': '8.0.0'}, 'timestamp': 1742304283170}})

In [320]:
route

{'duration': 205.4,
 'route': {'type': 'FeatureCollection',
  'bbox': [8.680677, 49.406906, 8.694327, 49.408968],
  'features': [{'bbox': [8.680677, 49.406906, 8.694327, 49.408968],
    'type': 'Feature',
    'properties': {'segments': [{'distance': 1295.8,
       'duration': 205.4,
       'steps': [{'distance': 866.2,
         'duration': 124.7,
         'type': 11,
         'instruction': 'Head east on Bergheimer Straße',
         'name': 'Bergheimer Straße',
         'way_points': [0, 15]},
        {'distance': 211.3,
         'duration': 36.0,
         'type': 1,
         'instruction': 'Turn right onto Rohrbacher Straße, B 3',
         'name': 'Rohrbacher Straße, B 3',
         'way_points': [15, 21]},
        {'distance': 175.4,
         'duration': 33.1,
         'type': 0,
         'instruction': 'Turn left onto Adenauerplatz, B 3',
         'name': 'Adenauerplatz, B 3',
         'way_points': [21, 30]},
        {'distance': 38.7,
         'duration': 8.6,
         'type': 0,
 

In [ ]:
# Compute routes from each valid point to the patient
# Compute routes from each valid point to the patient
routes = []
for _, row in df_filter_points.iterrows():
        route = directions([[8.680678,49.407841], [8.693779,49.407558]])
        routes.append(route)

ApiError: 500 ({'error': {'code': 2099, 'message': 'Unable to get an appropriate route profile for RoutePreference = foot-walking'}, 'info': {'engine': {'build_date': '2024-03-21T13:55:54Z', 'version': '8.0.0'}, 'timestamp': 1742302724457}})

In [117]:
routes

[{'duration': 258.1,
  'route': {'type': 'FeatureCollection',
   'bbox': [4.699047, 50.88762, 4.700195, 50.889371],
   'features': [{'bbox': [4.699047, 50.88762, 4.700195, 50.889371],
     'type': 'Feature',
     'properties': {'segments': [{'distance': 358.5,
        'duration': 258.1,
        'steps': [{'distance': 118.6,
          'duration': 85.4,
          'type': 11,
          'instruction': 'Head east',
          'name': '-',
          'way_points': [0, 8]},
         {'distance': 70.6,
          'duration': 50.8,
          'type': 2,
          'instruction': 'Turn sharp left',
          'name': '-',
          'way_points': [8, 12]},
         {'distance': 51.9,
          'duration': 37.4,
          'type': 1,
          'instruction': 'Turn right onto Tramweg',
          'name': 'Tramweg',
          'way_points': [12, 15]},
         {'distance': 45.8,
          'duration': 33.0,
          'type': 0,
          'instruction': 'Turn left',
          'name': '-',
          'way_points

In [123]:
# only include accepted AEDs
gdf_open = gdf.iloc[[0,1]].copy()
gdf_open

,lon,lat,geometry,AED
1,4.696771,50.882662,"POLYGON ((4.68680 50.88487, 4.68743 50.88203, ...",POINT (4.69677 50.88266)
2,4.705893,50.891915,"POLYGON ((4.69766 50.88972, 4.69971 50.88766, ...",POINT (4.70589 50.89192)


In [126]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15.4, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.15
    }
).add_to(m)

# # Add the patient marker (in red)
folium.CircleMarker(
    location=[patients[1], patients[0]],
    radius=7,  # Bigger circle
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(m)


folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa', icon_color='black')  # Red icon on white marker
).add_to(m)

# Add isochrones for AEDs (in blue)
# for _, row in gdf.iterrows():
#     folium.GeoJson(
#         row['geometry'],
#         style_function=lambda x: {
#             'fillColor': 'green',
#             'color': 'green',
#             'weight': 2,
#             'fillOpacity': 0.15
#         }
#     ).add_to(m)

# Add AED markers (in red)
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row['AED'].y, row['AED'].x],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
# for _,row in df_filter_points.iterrows():
#     folium.Marker(
#         location=[row['coordinates'].y, row['coordinates'].x],
#         popup="Valid Point",
#         icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
#     ).add_to(m)

#     folium.Marker(
#             location=[row['coordinates'].y, row['coordinates'].x],
#             popup=f"Time: {duration_min} min",
#             icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
#     ).add_to(m)
# # Show the map


# Add valid points and routes with labels
for route, (_, row) in zip(routes, df_filter_accepted.iterrows()):
    try:
        duration = route["duration"]  # Extract duration (in seconds)
        duration_min = round(duration / 60, 1)  # Convert to minutes
        
        # Draw route (Yellow)
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in route["coordinates"]],
            color="yellow",
            weight=2.5,
            opacity=1
        ).add_to(m)

        # Add marker for the AED/random point
        folium.Marker(
            location=[row['coordinates'].y, row['coordinates'].x],  # Correctly accessing geometry
            icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
        ).add_to(m)

        # Add permanent label using folium.DivIcon
        folium.Marker(
            location=[(row['coordinates'].y - 0.00022), row['coordinates'].x],  # Slightly offset to avoid overlap
            icon=folium.DivIcon(
                html=f"""<div style="font-size: 15px; color: yellow; background: rgba(255,255,255,0.0);
                         padding: 0px; border-radius: 3px; font-weight: bold;">{duration_min} min</div>"""
            )
        ).add_to(m)

    except Exception as e:
        print(f"⚠️ Error plotting route: {e}")

# Show map
m



In [147]:
# Filter points inside the isochrone polygon
# Convert Isochrone to Polygon

AED1_coordinate_tuples = [[(point)] for point in list(df_filter_accepted['coordinates']) if point.within(gdf.loc[1,'geometry'])]
df_filter_points_AED1 = pd.DataFrame(AED1_coordinate_tuples, columns = ['coordinates'])


In [148]:
print(df_filter_points_AED1)

                                   coordinates
0  POINT (4.694073171314823 50.88456620138093)
1  POINT (4.697383285811491 50.88792467151592)
2  POINT (4.699446605146951 50.88610535264766)


In [153]:
gdf_open.loc[2]

lon                                                  4.705893
lat                                                 50.891915
geometry    POLYGON ((4.697656 50.88972, 4.699708 50.88766...
AED                       POINT (4.7058934 50.89191539999999)
Name: 2, dtype: object

In [154]:
# Compute routes from each valid point to the patient
# Compute routes from each valid point to the patient
routes_AED1 = []
for _, row in df_filter_points_AED1.iterrows():
    try:
        route_1 = directions([[row['coordinates'].x, row['coordinates'].y],[gdf['lon'].loc[1],gdf['lat'].loc[1]], [patients[0], patients[1]]])
        routes_AED1.append(route_1)
    except:
        print(f"⚠️ Skipping point due to error")  # Log the error but continue

In [159]:
# Filter points inside the isochrone polygon
# Convert Isochrone to Polygon

AED2_coordinate_tuples = [[(point)] for point in list(df_filter_accepted['coordinates']) if point.within(gdf.loc[2,'geometry'])]
df_filter_points_AED2 = pd.DataFrame(AED2_coordinate_tuples, columns = ['coordinates'])

In [160]:
print(df_filter_points_AED2)

                                     coordinates
0   POINT (4.6988937307183765 50.88928789041148)
1  POINT (4.6983404708145855 50.891136735680725)
2    POINT (4.699814863109981 50.88963071924299)


In [161]:
# Compute routes from each valid point to the patient
# Compute routes from each valid point to the patient
routes_AED2 = []
for _, row in df_filter_points_AED2.iterrows():
    try:
        route_2 = directions([[row['coordinates'].x, row['coordinates'].y], [gdf_open['lon'].loc[2],gdf_open['lat'].loc[2]], [patients[0], patients[1]]])
        routes_AED2.append(route_2)
    except:
        print(f"⚠️ Skipping point due to error")  # Log the error but continue

In [168]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15.4, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.15
    }
).add_to(m)

# # Add the patient marker (in red)
folium.CircleMarker(
    location=[patients[1], patients[0]],
    radius=7,  # Bigger circle
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.6,
).add_to(m)


folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa', icon_color='black')  # Red icon on white marker
).add_to(m)

# Add isochrones for AEDs (in blue)
for _, row in gdf_open.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'green',
            'weight': 2,
            'fillOpacity': 0.15
        }
    ).add_to(m)

# Add AED markers (in red)
for _, row in gdf_open.iterrows():
    folium.Marker(
        location=[row['AED'].y, row['AED'].x],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
for _,row in df_filter_accepted.iterrows():
    folium.Marker(
        location=[row['coordinates'].y, row['coordinates'].x],
        popup="Valid Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

    folium.Marker(
            location=[row['coordinates'].y, row['coordinates'].x],
            popup=f"Time: {duration_min} min",
            icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)
# Show the map


# Add valid points and routes with labels
for route, (_, row) in zip(routes, df_filter_accepted.iterrows()):
    try:
        duration = route["duration"]  # Extract duration (in seconds)
        duration_min = round(duration / 60, 1)  # Convert to minutes
      
        # Draw route (Yellow)
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in route["coordinates"]],
            color="yellow",
            weight=2.5,
            opacity=1
        ).add_to(m)
         # Add marker for the AED/random point
        folium.Marker(
            location=[row['coordinates'].y, row['coordinates'].x],  # Correctly accessing geometry
            icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
        ).add_to(m)

#         # Add permanent label using folium.DivIcon
        folium.Marker(
            location=[(row['coordinates'].y - 0.00022), row['coordinates'].x - 0.0006],  # Slightly offset to avoid overlap
            icon=folium.DivIcon(
                html=f"""<div style="font-size: 15px; color: yellow; background: rgba(255,255,255,0.0);
                         padding: 0px; border-radius: 3px; font-weight: bold;">{duration_min} min</div>"""
            )
        ).add_to(m)

    except Exception as e:
        print(f"⚠️ Error plotting route: {e}")

    # Add valid points and routes with labels
for route_1, (_, row) in zip(routes_AED1, df_filter_points_AED1.iterrows()):
    try:
        duration = route_1["duration"]  # Extract duration (in seconds)
        duration_min = round(duration / 60, 1)  # Convert to minutes
        
        # Draw route (Yellow)
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in route_1["coordinates"]],
            color="green",
            weight=2.5,
            opacity=1
        ).add_to(m)

        
        # Add permanent label using folium.DivIcon
        folium.Marker(
            location=[(row['coordinates'].y - 0.00022), row['coordinates'].x+ 0.0006],  # Slightly offset to avoid overlap
            icon=folium.DivIcon(
                html=f"""<div style="font-size: 15px; color: green; background: rgba(255,255,255,0.0);
                         padding: 0px; border-radius: 3px; font-weight: bold;">{duration_min} min</div>"""
            )
        ).add_to(m)

    except Exception as e:
        print(f"⚠️ Error plotting route: {e}")

    # Add valid points and routes with labels
for route_2, (_, row) in zip(routes_AED2, df_filter_points_AED2.iterrows()):
    try:
        duration = route_2["duration"]  # Extract duration (in seconds)
        duration_min = round(duration / 60, 1)  # Convert to minutes
        
        # Draw route (Yellow)
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in route_2["coordinates"]],
            color="red",
            weight=2.5,
            opacity=1
        ).add_to(m)

        
        # Add permanent label using folium.DivIcon
        folium.Marker(
            location=[(row['coordinates'].y - 0.00022), row['coordinates'].x+0.0006],  # Slightly offset to avoid overlap
            icon=folium.DivIcon(
                html=f"""<div style="font-size: 15px; color: red; background: rgba(255,255,255,0.0);
                         padding: 0px; border-radius: 3px; font-weight: bold;">{duration_min} min</div>"""
            )
        ).add_to(m)

    except Exception as e:
        print(f"⚠️ Error plotting route: {e}")

# Show map
m


In [176]:
route_1

{'duration': 302.9,
 'route': {'type': 'FeatureCollection',
  'bbox': [4.69727, 50.88762, 4.700195, 50.888625],
  'features': [{'bbox': [4.69727, 50.88762, 4.700195, 50.888625],
    'type': 'Feature',
    'properties': {'segments': [{'distance': 420.6,
       'duration': 302.9,
       'steps': [{'distance': 180.7,
         'duration': 130.1,
         'type': 11,
         'instruction': 'Head north',
         'name': '-',
         'way_points': [0, 9]},
        {'distance': 70.6,
         'duration': 50.8,
         'type': 13,
         'instruction': 'Keep right',
         'name': '-',
         'way_points': [9, 13]},
        {'distance': 51.9,
         'duration': 37.4,
         'type': 1,
         'instruction': 'Turn right onto Tramweg',
         'name': 'Tramweg',
         'way_points': [13, 16]},
        {'distance': 45.8,
         'duration': 33.0,
         'type': 0,
         'instruction': 'Turn left',
         'name': '-',
         'way_points': [16, 27]},
        {'distance': 

In [295]:
routes_AED1[0]['duration']

806.0

In [170]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15.4, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.15
    }
).add_to(m)

# Add the patient marker (in green)
folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa')
).add_to(m)

# Add isochrones for AEDs (in blue)
for _, row in gdf_open.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'green',
            'weight': 2,
            'fillOpacity': 0.15
        }
    ).add_to(m)

# Add AED markers (in red)
for _, row in gdf_open.iterrows():
    folium.Marker(
        location=[row['AED'].y, row['AED'].x],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
for _,row in df_filter_accepted.iterrows():
    folium.Marker(
        location=[row['coordinates'].y, row['coordinates'].x],
        popup="Valid Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

    folium.Marker(
            location=[row['coordinates'].y, row['coordinates'].x],
            popup=f"Time: {duration_min} min",
            icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)
# Show the map

# Add random points (blue)
# Add filtered random points (Blue)
# Find the route with the smallest duration
min_duration_direct = float('inf')
min_route_direct = None
min_row_direct = None

for route, (_, row) in zip(routes, df_filter_accepted.iterrows()):
    try:
        duration_direct = route["duration"]
        if duration_direct < min_duration:
            min_duration_direct = duration
            min_route_direct = route_1
            min_row_direct = row
    except Exception as e:
        print(f"⚠️ Error checking route duration: {e}")

# Plot ONLY the fastest route
if min_route_direct and min_row_direct is not None:
        duration_min = round(duration_direct / 60, 1)  # Convert to minutes
        
        # Draw route (Yellow)
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in route["coordinates"]],
            color="yellow",
            weight=2.5,
            opacity=1
        ).add_to(m)

        # Add marker for the AED/random point
        folium.Marker(
            location=[row['coordinates'].y, row['coordinates'].x],  # Correctly accessing geometry
            icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
        ).add_to(m)

        # Add permanent label using folium.DivIcon
        folium.Marker(
            location=[(row['coordinates'].y - 0.00022), row['coordinates'].x - 0.0006],  # Slightly offset to avoid overlap
            icon=folium.DivIcon(
                html=f"""<div style="font-size: 15px; color: yellow; background: rgba(255,255,255,0.0);
                         padding: 0px; border-radius: 3px; font-weight: bold;">{duration_min} min</div>"""
            )
        ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
# Find the route with the smallest duration
min_duration = float('inf')
min_route = None
min_row = None

for route_1, (_, row) in zip(routes_AED1, df_filter_points_AED1.iterrows()):
    try:
        duration = route_1["duration"]
        if duration < min_duration:
            min_duration = duration
            min_route = route_1
            min_row = row
    except Exception as e:
        print(f"⚠️ Error checking route duration: {e}")

# Plot ONLY the fastest route
if min_route and min_row is not None:
    duration_min = round(min_duration / 60, 1)

    # Draw the fastest route (green)
    folium.PolyLine(
        locations=[(coord[1], coord[0]) for coord in min_route["coordinates"]],
        color="green",
        weight=3.5,
        opacity=1
    ).add_to(m)

    # Add marker at the random point
    folium.Marker(
        location=[min_row['coordinates'].y, min_row['coordinates'].x],
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

    # Add permanent label for the duration
    folium.Marker(
        location=[(min_row['coordinates'].y - 0.00022), min_row['coordinates'].x],
        icon=folium.DivIcon(
            html=f"""<div style="font-size: 15px; color: green; background: rgba(255,255,255,0.0);
                     padding: 0px; border-radius: 3px; font-weight: bold;">{duration_min} min</div>"""
        )
    ).add_to(m)

m

In [175]:
df_filter_Vec.iloc[1]['coordinates'].x

4.7058934

## Route Vector 

In [206]:
# Compute routes from each valid point to the patient
# Compute routes from each valid point to the patient
routes_Vec = []
for _, row in df_filter_Vec.iterrows():
    try:
        routes3 = directions([[row['coordinates'].x, row['coordinates'].y], [patients[0], patients[1]]], profile = 'driving-car', sleep = 1.5)
        routes_Vec.append(routes3)
    except:
        print(f"⚠️ Skipping point due to error")  # Log the error but continue

In [211]:
routes3

{'duration': 715.0,
 'route': {'type': 'FeatureCollection',
  'bbox': [4.664313, 50.875947, 4.711286, 50.89373],
  'features': [{'bbox': [4.664313, 50.875947, 4.711286, 50.89373],
    'type': 'Feature',
    'properties': {'segments': [{'distance': 6997.9,
       'duration': 715.0,
       'steps': [{'distance': 93.4,
         'duration': 24.1,
         'type': 11,
         'instruction': 'Head south',
         'name': '-',
         'way_points': [0, 7]},
        {'distance': 478.6,
         'duration': 70.9,
         'type': 1,
         'instruction': 'Turn right onto Herestraat',
         'name': 'Herestraat',
         'way_points': [7, 21]},
        {'distance': 270.4,
         'duration': 47.3,
         'type': 7,
         'instruction': 'Enter the roundabout and take the 3rd exit onto Terbankstraat',
         'name': 'Terbankstraat',
         'exit_number': 3,
         'way_points': [21, 38]},
        {'distance': 101.3,
         'duration': 16.8,
         'type': 6,
         'instr

In [289]:
# Initialize a folium map centered around the patient’s location
m = folium.Map(location=[patients[1], patients[0]], zoom_start=15.4, tiles='CartoDB positron')

# Add the patient's isochrone (in red)
folium.GeoJson(
    iso_patient,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.15
    }
).add_to(m)

folium.GeoJson(
    iso_patient_vector,
    style_function=lambda x: {
        'fillColor': 'orange',
        'color': 'red',
        'weight': 2,
        'fillOpacity': 0.15
    }
).add_to(m)

# Add the patient marker (in green)
folium.Marker(
    location=[patients[1], patients[0]],
    popup="Patient",
    icon=folium.Icon(color='red', icon='user', prefix='fa')
).add_to(m)

# Add isochrones for AEDs (in blue)
for _, row in gdf_open.iterrows():
    folium.GeoJson(
        row['geometry'],
        style_function=lambda x: {
            'fillColor': 'green',
            'color': 'green',
            'weight': 2,
            'fillOpacity': 0.15
        }
    ).add_to(m)

# Add AED markers (in red)
for _, row in gdf_open.iterrows():
    folium.Marker(
        location=[row['AED'].y, row['AED'].x],
        popup="AED",
        icon=folium.Icon(color='green', icon='heartbeat', prefix='fa')
    ).add_to(m)

# Add AED markers (in red)
for _, row in df_filter_Vec.iterrows():
    folium.Marker(
        location=[row['coordinates'].y, row['coordinates'].x],
        popup="Vector",
        icon=folium.Icon(color='orange', icon='truck-medical', prefix='fa')
    ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
for _,row in df_filter_accepted.iterrows():
    folium.Marker(
        location=[row['coordinates'].y, row['coordinates'].x],
        popup="Valid Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

    folium.Marker(
            location=[row['coordinates'].y, row['coordinates'].x],
            popup=f"Time: {duration_min} min",
            icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)
# Show the map

# Add random points (blue)
# Add filtered random points (Blue)
# Find the route with the smallest duration
min_duration_direct = float('inf')
min_route_direct = None
min_row_direct = None

for route, (_, row) in zip(routes, df_filter_accepted.iterrows()):
    try:
        duration_direct = route["duration"]
        if duration_direct < min_duration:
            min_duration_direct = duration
            min_route_direct = route
            min_row_direct = row
    except Exception as e:
        print(f"⚠️ Error checking route duration: {e}")

# Plot ONLY the fastest route
if min_route_direct and min_row_direct is not None:
        duration_min = round(duration_direct / 60, 1)  # Convert to minutes
        
        # Draw route (Yellow)
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in route["coordinates"]],
            color="yellow",
            weight=2.5,
            opacity=1
        ).add_to(m)

        # Add marker for the AED/random point
        folium.Marker(
            location=[row['coordinates'].y, row['coordinates'].x],  # Correctly accessing geometry
            icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
        ).add_to(m)

        # Add permanent label using folium.DivIcon
        folium.Marker(
            location=[(row['coordinates'].y - 0.00022), row['coordinates'].x - 0.0006],  # Slightly offset to avoid overlap
            icon=folium.DivIcon(
                html=f"""<div style="font-size: 15px; color: yellow; background: rgba(255,255,255,0.0);
                         padding: 0px; border-radius: 3px; font-weight: bold;">{duration_min} min</div>"""
            )
        ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
# Find the route with the smallest duration
min_duration = float('inf')
min_route = None
min_row = None

for route_1, (_, row) in zip(routes_AED1, df_filter_points_AED1.iterrows()):
    try:
        duration = route_1["duration"]
        if duration < min_duration:
            min_duration = duration
            min_route = route_1
            min_row = row
    except Exception as e:
        print(f"⚠️ Error checking route duration: {e}")

# Plot ONLY the fastest route
if min_route and min_row is not None:
    duration_min = round(min_duration / 60, 1)

    # Draw the fastest route (green)
    folium.PolyLine(
        locations=[(coord[1], coord[0]) for coord in min_route["coordinates"]],
        color="green",
        weight=3.5,
        opacity=1
    ).add_to(m)

    # Add marker at the random point
    folium.Marker(
        location=[min_row['coordinates'].y, min_row['coordinates'].x],
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

    # Add permanent label for the duration
    folium.Marker(
        location=[(min_row['coordinates'].y - 0.00022), min_row['coordinates'].x],
        icon=folium.DivIcon(
            html=f"""<div style="font-size: 15px; color: green; background: rgba(255,255,255,0.0);
                     padding: 0px; border-radius: 3px; font-weight: bold;">{duration_min} min</div>"""
        )
    ).add_to(m)

# Add random points (blue)
# Add filtered random points (Blue)
for _,row in df_filter_accepted.iterrows():
    folium.Marker(
        location=[row['coordinates'].y, row['coordinates'].x],
        popup="Valid Point",
        icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)

    folium.Marker(
            location=[row['coordinates'].y, row['coordinates'].x],
            popup=f"Time: {duration_min} min",
            icon=folium.Icon(color='blue', icon='user-nurse', prefix='fa')
    ).add_to(m)
# Show the map

# Add random points (blue)
# Add filtered random points (Blue)
# Find the route with the smallest duration
min_duration_direct = float('inf')
min_route_direct = None
min_row_direct = None

for route_vector, (_, row) in zip(routes_Vec, df_filter_Vec.iterrows()):
    try:
        duration_direct = route_vector["duration"]
        if duration_direct < min_duration_direct:
            min_duration_direct = duration_direct
            min_route_direct = route_vector
            min_row_direct = row
    except Exception as e:
        print(f"⚠️ Error checking route duration: {e}")

# Plot ONLY the fastest route
if min_route_direct and min_row_direct is not None:
    duration_min = round(min_duration_direct / 60, 1)  # Convert to minutes
    
    # Draw route (Blue)
    folium.PolyLine(
        locations=[(coord[1], coord[0]) for coord in min_route_direct["coordinates"]],
        color="blue",
        weight=2.5,
        opacity=1
    ).add_to(m)

    # Add marker for the fastest random point
    folium.Marker(
        location=[min_row_direct['coordinates'].y, min_row_direct['coordinates'].x],
        icon=folium.Icon(color='orange', icon='truck-medical', prefix='fa')
    ).add_to(m)

    # Add permanent label
    folium.Marker(
        location=[(min_row_direct['coordinates'].y - 0.00022), min_row_direct['coordinates'].x - 0.0006],
        icon=folium.DivIcon(
            html=f"""<div style="font-size: 15px; color: black; background: rgba(255,255,255,0.0);
                     padding: 0px; border-radius: 3px; font-weight: bold;">{duration_min} min</div>"""
        )
    ).add_to(m)


m

In [247]:
df_filter_Vec

,coordinates
0,POINT (4.7243436 50.8696549)
1,POINT (4.6953981 50.8785355)
2,POINT (4.6658621 50.8774603)
3,POINT (4.6998331 50.8723536)
4,POINT (4.6731993 50.8780791)


In [243]:
min_duration = 700
for route_vector, (_, row) in zip(routes_Vec, df_filter_Vec.iterrows()):
    try:
        duration_direct = route_vector["duration"]
        if duration_direct < min_duration:
            min_duration_direct = duration
            min_route_direct = route_vector
            min_row_direct = row
    except Exception as e:
        print(f"⚠️ Error checking route duration: {e}")

In [266]:
len(routes_Vec)

5

In [274]:
routes_Vec_slice = routes_Vec[1:2]

In [275]:
len(routes_Vec_slice)

1

In [276]:
df_filter_Vec_slie = df_filter_Vec.iloc[1:2].copy()

In [246]:
df_filter_Vec

,coordinates
0,POINT (4.7243436 50.8696549)
1,POINT (4.6953981 50.8785355)
2,POINT (4.6658621 50.8774603)
3,POINT (4.6998331 50.8723536)
4,POINT (4.6731993 50.8780791)


In [245]:
df_filter_Vec.iloc[2]['coordinates'].y

50.8774603

In [285]:
routes_test_3 = directions([[df_filter_Vec.iloc[3]['coordinates'].x, df_filter_Vec.iloc[3]['coordinates'].y], [patients[0], patients[1]]], profile = 'driving-car', sleep = 1.5)

In [286]:
routes_test_3

{'duration': 731.2,
 'route': {'type': 'FeatureCollection',
  'bbox': [4.690681, 50.868755, 4.702694, 50.88762],
  'features': [{'bbox': [4.690681, 50.868755, 4.702694, 50.88762],
    'type': 'Feature',
    'properties': {'segments': [{'distance': 3917.0,
       'duration': 731.2,
       'steps': [{'distance': 6.3,
         'duration': 4.5,
         'type': 11,
         'instruction': 'Head west',
         'name': '-',
         'way_points': [0, 1]},
        {'distance': 401.7,
         'duration': 82.7,
         'type': 0,
         'instruction': 'Turn left onto Naamsestraat',
         'name': 'Naamsestraat',
         'way_points': [1, 12]},
        {'distance': 692.8,
         'duration': 91.4,
         'type': 1,
         'instruction': 'Turn right onto Tervuursevest, R23',
         'name': 'Tervuursevest, R23',
         'way_points': [12, 30]},
        {'distance': 972.6,
         'duration': 154.7,
         'type': 1,
         'instruction': 'Turn right onto Kapucijnenvoer',
     

In [234]:
routes_Vec

[{'duration': 756.1,
  'route': {'type': 'FeatureCollection',
   'bbox': [4.699319, 50.868531, 4.725842, 50.88762],
   'features': [{'bbox': [4.699319, 50.868531, 4.725842, 50.88762],
     'type': 'Feature',
     'properties': {'segments': [{'distance': 4641.0,
        'duration': 756.1,
        'steps': [{'distance': 855.6,
          'duration': 205.4,
          'type': 11,
          'instruction': 'Head southeast on Korbeek-Losestraat',
          'name': 'Korbeek-Losestraat',
          'way_points': [0, 21]},
         {'distance': 82.5,
          'duration': 15.8,
          'type': 2,
          'instruction': 'Turn sharp left onto Tiensesteenweg, N3',
          'name': 'Tiensesteenweg, N3',
          'way_points': [21, 23]},
         {'distance': 1308.1,
          'duration': 163.8,
          'type': 1,
          'instruction': 'Turn right onto Martelarenlaan',
          'name': 'Martelarenlaan',
          'way_points': [23, 53]},
         {'distance': 706.7,
          'duration': 89